In [0]:
#Biblioteca
from pyspark.sql import functions as F
import os

In [0]:
# Variaveis e locais para salvar ou consumi dados.
CATALOG = "bh"
SCHEMA  = "mobilidade"
VOLUME  = "data_mobility_bh"

BASE_PATH = f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}"
SILVER_DIR = f"{BASE_PATH}/silver/mco"
GOLD_DIR   = f"{BASE_PATH}/gold/mco_kpis"

In [0]:
#Ler tabela Silver

df_silver = spark.read.format("delta").load(SILVER_DIR)

In [0]:
#KPIs simples: Count, Count distinct e horário que foi gerado. 
df_gold = (
    df_silver
      .agg(
          F.count("*").alias("total_registros"),
          F.countDistinct(df_silver.columns[0]).alias("valores_distintos_primeira_coluna")
      )
      .withColumn("gerado_em", F.current_timestamp())
)

In [0]:
# Verifica existencia do destinatário Gold e salvar camada Gold
os.makedirs(GOLD_DIR, exist_ok=True)

(df_gold.write
    .format("delta")
    .mode("overwrite")
    .save(GOLD_DIR)
)

In [0]:
# Visualizar dados da camada Gold
display(df_gold)